# Prompt Engineering Workshop

## Installieren der notwendigen Bibliotheken

In [ ]:
!git clone "https://github.com/GDSC-TU-Berlin/prompt_engineering.git"
!mkdir stupo
!mv prompt_engineering/stupo/embeddings.npz stupo/embeddings.npz

In [ ]:
!pip install openai numpy

In [ ]:
from openai import OpenAI
import json
import prompt_engineering.tests as tests
import prompt_engineering.utils as utils

In [ ]:
client = OpenAI(
    api_key="",
)

## Hallo OpenAI
In unserem ersten Beispiel wollen wir uns anschauen, wie wir OpenAI nutzen können um einen Text zu vervollständigen.

In [ ]:
response = None
print(response)

Wie wir sehen können gibt uns OpenAI eine Antwort zurück. Diese enthält insbesondere den vervollständigten Prompt. Wir können uns auch nur den vervollständigten Prompt ausgeben lassen mit:

In [ ]:
print(response.choices[0].text)

## Hauptstädte vervollständigen
In unserer ersten Aufgabe wollen wir eine Funktion schreiben, die uns die Hauptstadt eines Landes zurückgibt.
Dafür wollen wir eine Methode get_capital(country) nutzen, welche als Eingabe eine Text country bekommt und als Ausgabe NUR die Hauptstadt zurückgibt.

Zum Beispiel solle der Aufruf von get_capital("Deutschland") als Antwort "Berlin" liefern.

In [ ]:
def get_capital(country):
    response = client.completion.create(
        model="gpt-3.5-turbo-instruct"
    )
    return response.choices[0].text.strip()

In [ ]:
print(get_capital("Deutschland"))

In [ ]:
tests.test_get_capital(get_capital)

Wenn wir einfach nur die Frage stellen wie wir es in ChatGPT tun würden, bekommen wir leider nicht nur den Namen der Hauptstadt zurück...

Zum Beispiel liefert uns der Aufruf von get_capital("Deutschland") als Antwort "Die Hauptstadt von Deutschland ist Berlin." zurück. Wir wollen aber nur "Berlin" zurückbekommen.

In [ ]:
def get_capital(country):
    response = client.completion.create(
        model="gpt-3.5-turbo-instruct"
    )
    return response.choices[0].text.strip()

In [ ]:
print(get_capital("Deutschland"))

In [ ]:
tests.test_get_capital(get_capital)

### Verbessern der Antwort durch Beispiele
LLMs sind sehr gut darin, Mustern zu erkennen und zu folgen. Wir können dies nutzen, um die Qualität der Antworten zu verbessern. Dafür können wir Beispiele nutzen, die wir dem LLM geben. Diese Beispiele werden dann in die Berechnung der Antwort mit einbezogen.

In [ ]:
def get_capital(country):
    response = client.completion.create(
        model="gpt-3.5-turbo-instruct"
    )
    return response.choices[0].text.strip()

In [ ]:
tests.test_get_capital(get_capital)

### Chatmodels
Wir wollen uns nun anschauen, wie wir das Problem des Hauptstädte findens mit Hilfe eines Chatmodels lösen können.

In [ ]:
def get_capital(country):
    response = None
    return response.choices[0].message.content.strip()

In [ ]:
tests.test_get_capital(get_capital)

### Vokabeln lernen
Wir wollen nun eine etwas sinnvolleren Use-Case betrachten.

Angenommen wir entwickeln eine App zum lern von English Vokabeln. Hierfür wollen wir eine Methode schreiben, die ein Deutsches Wort als Eingabe bekommt und folgende Ausgaben liefert:
- Die englische Übersetzung des Wortes
- Die Definition des Wortes
- Ein Beispiel für die Verwendung des Wortes in einem Satz
- Die Wortart des Wortes

Dafür wollen wir eine Methode get_english_translation entwickeln welche uns die Information als JSON zurückgibt.
 Ein Beispiel für die Ausgabe ist:
 ```
    {
        "German": "Haus",
        "English_Translation": "House",
        "Definition": "A building for human habitation, especially one that consists of a ground floor and one or more upper storeys.",
        "Example_Sentence": "The family lives in a beautiful house with a big garden.",
        "Part_of_Speech": "Noun"
    }
 ```

In [ ]:
example_output = ("{"
                  "    \"German\": \"Haus\","
                  "    \"English_Translation\": \"House\","
                  "    \"Definition\": \"A building for human habitation, especially one that consists of a ground floor and one or more upper storeys.\","
                  "    \"Example_Sentence\": \"The family lives in a beautiful house with a big garden.\","
                  "    \"Part_of_Speech\": \"Noun\""
                  "}")


def get_english_translation(word):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
    )
    return json.loads(response.choices[0].message.content.strip())

In [ ]:
get_english_translation("Uhr")

## Chatmodel mit eigenen Informationen
Sehr häufiges werden LLM einsetzten um Chatmodele zu realisieren die Antworten geben, basierend auf eigenen Informationen. Zum Beispiel im Unternehmenskontext, um Kundenanfragen zu beantworten.

Wir wollen uns nun anschauen, wie wir ein Chatbot erstellen können, der eigenes Wissen nutzt. Als Beispiel dafür wollen wir einen Chatbot erstellen, der in der Lage ist Fragen zur Informatik StuPO der TU Berlin zu beantworten.

In [ ]:
def answer_question(question):
    context = utils.get_stupo_info(question, client)
    info = "\n".join(context)
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
    )
    return response.choices[0].message.content.strip()

In [ ]:
question = "Wie viele Semester hat das Studium?"
print(answer_question(question))

## Kreatives Schreiben
In dieser Aufgabe wollen wir uns Anschauen wie die Temperatur die Kreativität und Qualität der Antworten beeinflussen.

Dafür wollen wir eine Methode geschichten_schreiber schreiben, welche als Eingabe ein Thema bekommt und als Ausgabe eine kurze Geschichte über das Thema zurückgibt.

In [ ]:
def geschichten_schreiber(topic):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "user",
                "content": f"Schreibe mir eine kurze Geschichte über {topic}."
            },
        ],
    )
    return response.choices[0].message.content.strip()

In [ ]:
print(geschichten_schreiber("einen Hund"))

## Knobel Aufgaben mit Chain of Thoughts
Eine sehr häufige Anwendung von LLM ist das Lösen von Knobel Aufgaben. Wir wollen uns nun anschauen, wie wir Knobel Aufgaben mit Hilfe von Chain of Thoughts lösen können und ob wir damit bessere Ergebnisse erzielen können.


In [ ]:
def solve_knobel_aufgabe(aufgabe):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "user",
                "content": f"{aufgabe}"
            },
        ]
    )
    return response.choices[0].message.content.strip()

In [ ]:
aufgabe = "Du schaust auf ein Portrait und ich sage Dir: Der Vater der Person auf dem Potrait ist der Sohn meines Vaters, aber ich habe keine Geschwister. Wessen Bild schaust Du an?"

print(solve_knobel_aufgabe(aufgabe))

In [ ]:
tests.test_knobel_aufgaben(solve_knobel_aufgabe, client, rep=5)

## Systemnachrichten
Neben den Usernachrichten und Assistennachichten können wir auch Systemnachrichten nutzen. Diese dienen dazu dem System zu erklären, wie es sich verhalten soll.

Wir wollen damit einen Chatbot erstellen, der immer unhöflich antwortet.

In [ ]:
def bad_gpt(question):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "user",
                "content": question
            },
        ]
    )
    return response.choices[0].message.content.strip()

In [ ]:
print(bad_gpt("Was ist die Hauptstadt von Deutschland?"))

## Knobel Aufgaben mit Chain of Thoughts und Self Consistency
Wir haben uns bereits angeschaut, wie wir Knobel Aufgaben mit Chain of Thoughts lösen können. Wir haben dabei aber festgestellt das selbst bei GPT-4 die Antworten nicht immer deterministisch sind und es teilweise falsche Antworten gibt. 

Ein Trick um dies zu verhindern ist es, die Antworten mit Self Consistency zu kombinieren. Dabei wird die Antwort mehrfach generiert und die Antwort die am häufigsten vorkommt wird als Antwort genommen.

In [ ]:
def solve_knobel_aufgabe_advances(aufgabe):
    responses = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "user",
                "content": f"{aufgabe} Denken wir Schritt für Schritt:"
            },
        ]
    )
    
    return response.choices[0].message.content.strip()